In [ ]:
#https://www.cockroachlabs.com/docs/stable/start-a-local-cluster-in-docker-linux.html#step-7-stop-the-cluster

In [1]:
import json

In [2]:
from sqlalchemy import create_engine, Column, String, Integer, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm

In [3]:
import ray

In [4]:
%%time
with open('13_266069_040_003 L02 PAS.json', 'r') as file:
    # Load the JSON data into a Python dictionary
    data = json.load(file)

In [5]:
import shapely
from shapely.geometry import shape

In [6]:
# Define the base class for ORM
Base = declarative_base()

# Define the Annotation class
class Annotation(Base):
    __tablename__ = 'annotations'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    color = Column(ARRAY(Integer))
    coordinates = Column(Geometry('POLYGON'))

In [9]:
#%%timeit
# Create a connection to CockroachDB
#engine = create_engine('cockroachdb://root@170.140.138.73:26257/test1',echo=True)
engine = create_engine('cockroachdb://root@170.140.138.87:26257/test1')#,echo=True)

In [ ]:
from sqlalchemy_utils import database_exists, create_database

if not database_exists(engine.url):
    print("creating")
    create_database(engine.url)

print(database_exists(engine.url))

In [19]:
# Create the table if it doesn't exist
Base.metadata.create_all(engine)

In [15]:
@ray.remote
def commit(annos):
    from sqlalchemy import create_engine, Column, String, Integer, func
    from sqlalchemy import create_engine, Column, String, Integer, func
    from sqlalchemy.ext.declarative import declarative_base
    from sqlalchemy.dialects.postgresql import ARRAY
    from sqlalchemy.orm import sessionmaker
    from geoalchemy2 import Geometry 
    from tqdm import tqdm
    import numpy as np
# Define the base class for ORM
    Base = declarative_base()
    
    # Define the Annotation class
    class Annotation(Base):
        __tablename__ = 'annotations'
    
        id = Column(Integer, primary_key=True)
        name = Column(String)
        color = Column(ARRAY(Integer))
        coordinates = Column(Geometry('POLYGON'))
    
    #engine = create_engine('cockroachdb://root@170.140.138.87:26257/test1')#,echo=True)
    db_url=np.random.choice(['cockroachdb://root@170.140.138.87:26257/test1',
                             'cockroachdb://root@170.140.138.63:26257/test1',
                             'cockroachdb://root@170.140.138.73:26257/test1'])
    engine = create_engine(db_url)#,echo=True)
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
    for a in annos:
        session.add(Annotation(name=a[0], color=a[1], coordinates=a[2].wkt))
    
    session.commit()
    session.close()

In [12]:
ray.init()

In [22]:
%%time
#--- reasonable

tasks = []
all_anno=[]
for i,d in enumerate(tqdm(data)):
    shapely_geometry = shape(d['geometry'])
    # new_annotation = Annotation(name=d['properties']['classification']['name'],
    #                             color=d['properties']['classification']['color'], 
    #                             coordinates=shapely_geometry.wkt)
    name = d['properties']['classification']['name']
    color = d['properties']['classification']['color']
    geom = shapely_geometry
    
    all_anno.append([name,color,geom])
    if len(all_anno)==1_000:
        task= commit.remote(all_anno)
        tasks.append(task)
        all_anno=[]
        
    

print("waiting")
vals=ray.get(tasks)
print(len(vals))



In [ ]:

import pickle

In [ ]:
a=pickle.dumps(new_annotation)

In [ ]:
#---------- below is scratch code for testing query

In [ ]:
from shapely.geometry import Point
from geoalchemy2.functions import ST_Contains, ST_Intersects, ST_AsBinary, ST_AsText
from geoalchemy2 import WKTElement

In [ ]:
import numpy as np

In [ ]:
query_geom = shape(data[0]['geometry'])
#query_geom = shape(np.random.choice(data)['geometry'])

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
#%%timeit

query_geom = shape(np.random.choice(data)['geometry'])
results = session.query(Annotation).filter(ST_Contains(Annotation.coordinates, query_geom.centroid.wkt)).all()

In [ ]:
# Print the results
from shapely import wkt, wkb
for i,annotation in enumerate(results):
    print(i,annotation.name)
    shapely_obj=wkb.loads(str(annotation.coordinates))
    print(shapely_obj.centroid)

In [ ]:
shapely_obj

In [ ]:

shapely_obj.centroid.y

In [ ]:
#--- below is scratch code for testing converting

In [ ]:
from shapely.geometry import mapping

In [ ]:
geo=mapping(shapely_obj)

In [ ]:
import geojson

In [ ]:

props={"color":[1,2,3],"name":"tubule"}

In [ ]:
props

In [ ]:
feature = geojson.Feature(geometry=geo, properties=props)

In [ ]:
geojson.dumps(feature)